## Classifying Crops by Inception V3, VGG16 and own model using Keras

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# import  libraries 

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

# DATA separating and resize for train and test

In [4]:
# re-size 
IMAGE_SIZE = [224, 224]

train_path = 'train'
valid_path = 'test'


In [5]:
#Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 5028 images belonging to 6 classes.


In [7]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 156 images belonging to 6 classes.


In [8]:
from keras.metrics import *
from keras.callbacks import LearningRateScheduler


METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
]

# Inception V3

In [15]:
# Import the InceptionV3 library 
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [22]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [23]:
# our layers 
x = Flatten()(inception.output)

In [24]:
prediction = Dense(6, activation='softmax')(x)

# create a model object
model1 = Model(inputs=inception.input, outputs=prediction)

In [25]:
# telling the model what cost and optimization method to use
model1.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=METRICS
)


In [26]:
# # fit the model

r = model1.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20


<ipython-input-26-f327ae07ac14>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model1.fit_generator(


315/315 [==============================] - 228s 708ms/step - loss: 3.4700 - tp: 3243.0000 - fp: 1735.0000 - tn: 23405.0000 - fn: 1785.0000 - accuracy: 0.8833 - precision: 0.6515 - recall: 0.6450 - val_loss: 3.7050 - val_tp: 108.0000 - val_fp: 47.0000 - val_tn: 733.0000 - val_fn: 48.0000 - val_accuracy: 0.8985 - val_precision: 0.6968 - val_recall: 0.6923
Epoch 2/20
315/315 [==============================] - 225s 713ms/step - loss: 3.2860 - tp: 3559.0000 - fp: 1458.0000 - tn: 23682.0000 - fn: 1469.0000 - accuracy: 0.9030 - precision: 0.7094 - recall: 0.7078 - val_loss: 3.5319 - val_tp: 121.0000 - val_fp: 35.0000 - val_tn: 745.0000 - val_fn: 35.0000 - val_accuracy: 0.9252 - val_precision: 0.7756 - val_recall: 0.7756
Epoch 3/20
315/315 [==============================] - 224s 711ms/step - loss: 3.2697 - tp: 3728.0000 - fp: 1285.0000 - tn: 23855.0000 - fn: 1300.0000 - accuracy: 0.9143 - precision: 0.7437 - recall: 0.7414 - val_loss: 6.3977 - val_tp: 106.0000 - val_fp: 50.0000 - val_tn: 730.0

In [27]:
# model1.save('inception_v3.h5')

# USING VGG16


In [29]:
import tensorflow as tf
import os
import numpy as np

In [30]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential 
from glob import glob

In [31]:
Image_size=[224,224]
vgg=VGG16(input_shape=Image_size+[3],weights='imagenet',include_top=False)
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [32]:
for layer in vgg.layers:
    layer.trainable=False

In [33]:
x=Flatten()(vgg.output)
prediction=Dense(6,activation='softmax')(x)
model2=Model(inputs=vgg.input,outputs=prediction)
model2.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [34]:
model2.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [35]:
# # fit the model

r = model2.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20


<ipython-input-35-258b1d4b3a8d>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model2.fit_generator(


315/315 [==============================] - 836s 3s/step - loss: 0.8519 - accuracy: 0.6611 - val_loss: 0.6479 - val_accuracy: 0.7372
Epoch 2/20
315/315 [==============================] - 834s 3s/step - loss: 0.7029 - accuracy: 0.7311 - val_loss: 0.5311 - val_accuracy: 0.7949
Epoch 3/20
315/315 [==============================] - 835s 3s/step - loss: 0.6155 - accuracy: 0.7613 - val_loss: 0.5486 - val_accuracy: 0.7692
Epoch 4/20
315/315 [==============================] - 832s 3s/step - loss: 0.5708 - accuracy: 0.7812 - val_loss: 0.5961 - val_accuracy: 0.7756
Epoch 5/20
315/315 [==============================] - 832s 3s/step - loss: 0.5206 - accuracy: 0.8003 - val_loss: 0.6490 - val_accuracy: 0.7756
Epoch 6/20
315/315 [==============================] - 835s 3s/step - loss: 0.5312 - accuracy: 0.7977 - val_loss: 0.4885 - val_accuracy: 0.8205
Epoch 7/20
315/315 [==============================] - 837s 3s/step - loss: 0.4866 - accuracy: 0.8125 - val_loss: 0.4402 - val_accuracy: 0.8333
Epoch 8/20

In [36]:
# model2.save('vgg16.h5')

# Using own CNN model

In [38]:
# I have used 3 Convolutional layers followed by max-pooling layers, a drop layer and 2 dense layer
model3 = Sequential()
model3.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
model3.add(MaxPool2D())


model3.add(Conv2D(32, 3, padding="same", activation="relu"))
model3.add(MaxPool2D())


model3.add(Conv2D(64, 3, padding="same", activation="relu"))
model3.add(MaxPool2D())
model3.add(Dropout(0.4))


model3.add(Flatten())
model3.add(Dense(128,activation="relu"))
model3.add(Dense(6, activation="softmax"))

model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_94 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_95 (Conv2D)          (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_96 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 28, 28, 64)       0

In [39]:
model3.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [40]:
# # fit the model

r = model3.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-40-7a1f4646b6bc>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model3.fit_generator(


Epoch 1/20
315/315 [==============================] - 195s 617ms/step - loss: 1.2920 - accuracy: 0.4914 - val_loss: 0.7418 - val_accuracy: 0.6474
Epoch 2/20
315/315 [==============================] - 191s 607ms/step - loss: 0.9562 - accuracy: 0.6104 - val_loss: 0.6277 - val_accuracy: 0.6923
Epoch 3/20
315/315 [==============================] - 192s 609ms/step - loss: 0.8765 - accuracy: 0.6450 - val_loss: 0.6083 - val_accuracy: 0.7372
Epoch 4/20
315/315 [==============================] - 191s 607ms/step - loss: 0.8288 - accuracy: 0.6683 - val_loss: 0.6101 - val_accuracy: 0.7244
Epoch 5/20
315/315 [==============================] - 191s 606ms/step - loss: 0.7831 - accuracy: 0.6780 - val_loss: 0.5563 - val_accuracy: 0.7564
Epoch 6/20
315/315 [==============================] - 191s 607ms/step - loss: 0.7394 - accuracy: 0.7078 - val_loss: 0.5810 - val_accuracy: 0.7308
Epoch 7/20
315/315 [==============================] - 191s 606ms/step - loss: 0.7181 - accuracy: 0.7011 - val_loss: 0.5635 -

In [41]:
# model3.save('cnn.h5')

# Mobile NET V2

In [43]:
model4 = Sequential()
model4.add(tf.keras.applications.MobileNetV2(include_top=False, pooling = 'avg', weights='imagenet',input_shape=(224, 224, 3), classes=2))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(6, activation='sigmoid'))
model4.layers[0].trainable = False

In [44]:
# telling the model what cost and optimization method to use
model4.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=METRICS
)


In [45]:
# # fit the model

r = model4.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-45-6ef08da00759>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model4.fit_generator(


Epoch 1/20
315/315 [==============================] - 130s 403ms/step - loss: 0.8627 - tp: 3687.0000 - fp: 4823.0000 - tn: 21097.0000 - fn: 1497.0000 - accuracy: 0.7968 - precision: 0.4333 - recall: 0.7112 - val_loss: 0.7220 - val_tp: 105.0000 - val_fp: 89.0000 - val_tn: 691.0000 - val_fn: 51.0000 - val_accuracy: 0.8504 - val_precision: 0.5412 - val_recall: 0.6731
Epoch 2/20
315/315 [==============================] - 127s 402ms/step - loss: 0.6726 - tp: 3568.0000 - fp: 3047.0000 - tn: 22093.0000 - fn: 1460.0000 - accuracy: 0.8506 - precision: 0.5394 - recall: 0.7096 - val_loss: 0.6660 - val_tp: 118.0000 - val_fp: 96.0000 - val_tn: 684.0000 - val_fn: 38.0000 - val_accuracy: 0.8568 - val_precision: 0.5514 - val_recall: 0.7564
Epoch 3/20
315/315 [==============================] - 127s 401ms/step - loss: 0.6265 - tp: 3566.0000 - fp: 2514.0000 - tn: 22626.0000 - fn: 1462.0000 - accuracy: 0.8682 - precision: 0.5865 - recall: 0.7092 - val_loss: 0.6934 - val_tp: 121.0000 - val_fp: 82.0000 - va

In [46]:
# model4.save('mobile_net_v2.h5')

# DenseNet

In [16]:
from keras.applications.densenet import DenseNet201, DenseNet121

In [17]:
model5 = Sequential()
model5.add(DenseNet121(include_top=False, pooling = 'avg', weights='imagenet',input_shape=(224, 224, 3), classes=2))
model5.add(Dense(512, activation='relu'))
model5.add(Dense(128, activation='relu'))
model5.add(Dense(64, activation='relu'))
model5.add(Dense(6, activation='sigmoid'))
model5.layers[0].trainable = False


In [18]:
# telling the model what cost and optimization method to use
model5.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=METRICS
)


In [19]:
# fit the model

r = model5.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-19-aa5eccc02aa8>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model5.fit_generator(


Epoch 1/20
315/315 [==============================] - 442s 1s/step - loss: 0.7930 - tp: 4819.0000 - fp: 8224.0000 - tn: 16916.0000 - fn: 209.0000 - accuracy: 0.7205 - precision: 0.3695 - recall: 0.9584 - val_loss: 0.7201 - val_tp: 151.0000 - val_fp: 208.0000 - val_tn: 572.0000 - val_fn: 5.0000 - val_accuracy: 0.7724 - val_precision: 0.4206 - val_recall: 0.9679
Epoch 2/20
315/315 [==============================] - 422s 1s/step - loss: 0.6600 - tp: 4957.0000 - fp: 7938.0000 - tn: 17202.0000 - fn: 71.0000 - accuracy: 0.7345 - precision: 0.3844 - recall: 0.9859 - val_loss: 0.6447 - val_tp: 156.0000 - val_fp: 224.0000 - val_tn: 556.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.7607 - val_precision: 0.4105 - val_recall: 1.0000
Epoch 3/20
315/315 [==============================] - 422s 1s/step - loss: 0.6142 - tp: 4943.0000 - fp: 7499.0000 - tn: 17641.0000 - fn: 85.0000 - accuracy: 0.7486 - precision: 0.3973 - recall: 0.9831 - val_loss: 0.7096 - val_tp: 152.0000 - val_fp: 218.0000 - val_tn: 562

In [20]:
# model5.save('densenet.h5')

# EfficientB7

In [9]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input

In [10]:
base_model = EfficientNetB7(include_top=False, weights='imagenet', pooling='max', input_shape=(224,224,3))
base_model.trainable = False

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of cbcfe4450ddf6f3ad90b1b398090fe4a so we will re-download the data.
258088960/258076736 [==============================] - 240s 1us/step


In [11]:
x=Flatten()(base_model.output)
prediction=Dense(6,activation='softmax')(x)
model6=Model(inputs=base_model.input,outputs=prediction)
model6.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0]']      

                                                                                                  
 block1c_bn (BatchNormalization  (None, 112, 112, 32  128        ['block1c_dwconv[0][0]']         
 )                              )                                                                 
                                                                                                  
 block1c_activation (Activation  (None, 112, 112, 32  0          ['block1c_bn[0][0]']             
 )                              )                                                                 
                                                                                                  
 block1c_se_squeeze (GlobalAver  (None, 32)          0           ['block1c_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block1c_s

 )                                                                                                
                                                                                                  
 block2a_se_squeeze (GlobalAver  (None, 192)         0           ['block2a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block2a_se_reshape (Reshape)   (None, 1, 1, 192)    0           ['block2a_se_squeeze[0][0]']     
                                                                                                  
 block2a_se_reduce (Conv2D)     (None, 1, 1, 8)      1544        ['block2a_se_reshape[0][0]']     
                                                                                                  
 block2a_se_expand (Conv2D)     (None, 1, 1, 192)    1728        ['block2a_se_reduce[0][0]']      
          

                                                                                                  
 block2c_se_excite (Multiply)   (None, 56, 56, 288)  0           ['block2c_activation[0][0]',     
                                                                  'block2c_se_expand[0][0]']      
                                                                                                  
 block2c_project_conv (Conv2D)  (None, 56, 56, 48)   13824       ['block2c_se_excite[0][0]']      
                                                                                                  
 block2c_project_bn (BatchNorma  (None, 56, 56, 48)  192         ['block2c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2c_drop (Dropout)         (None, 56, 56, 48)   0           ['block2c_project_bn[0][0]']     
          

                                                                                                  
 block2e_project_bn (BatchNorma  (None, 56, 56, 48)  192         ['block2e_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2e_drop (Dropout)         (None, 56, 56, 48)   0           ['block2e_project_bn[0][0]']     
                                                                                                  
 block2e_add (Add)              (None, 56, 56, 48)   0           ['block2e_drop[0][0]',           
                                                                  'block2d_add[0][0]']            
                                                                                                  
 block2f_expand_conv (Conv2D)   (None, 56, 56, 288)  13824       ['block2e_add[0][0]']            
          

...

                                                                                                  
 block6h_expand_conv (Conv2D)   (None, 7, 7, 2304)   884736      ['block6g_add[0][0]']            
                                                                                                  
 block6h_expand_bn (BatchNormal  (None, 7, 7, 2304)  9216        ['block6h_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6h_expand_activation (Act  (None, 7, 7, 2304)  0           ['block6h_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6h_dwconv (DepthwiseConv2  (None, 7, 7, 2304)  57600       ['block6h_expand_activation[0][0]
 D)       

                                                                                                  
 block6j_expand_activation (Act  (None, 7, 7, 2304)  0           ['block6j_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6j_dwconv (DepthwiseConv2  (None, 7, 7, 2304)  57600       ['block6j_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block6j_bn (BatchNormalization  (None, 7, 7, 2304)  9216        ['block6j_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block6j_a

 D)                                                              ']                               
                                                                                                  
 block6l_bn (BatchNormalization  (None, 7, 7, 2304)  9216        ['block6l_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block6l_activation (Activation  (None, 7, 7, 2304)  0           ['block6l_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block6l_se_squeeze (GlobalAver  (None, 2304)        0           ['block6l_activation[0][0]']     
 agePooling2D)                                                                                    
          

 block7a_activation (Activation  (None, 7, 7, 2304)  0           ['block7a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block7a_se_squeeze (GlobalAver  (None, 2304)        0           ['block7a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block7a_se_reshape (Reshape)   (None, 1, 1, 2304)   0           ['block7a_se_squeeze[0][0]']     
                                                                                                  
 block7a_se_reduce (Conv2D)     (None, 1, 1, 96)     221280      ['block7a_se_reshape[0][0]']     
                                                                                                  
 block7a_s

 block7c_se_expand (Conv2D)     (None, 1, 1, 3840)   618240      ['block7c_se_reduce[0][0]']      
                                                                                                  
 block7c_se_excite (Multiply)   (None, 7, 7, 3840)   0           ['block7c_activation[0][0]',     
                                                                  'block7c_se_expand[0][0]']      
                                                                                                  
 block7c_project_conv (Conv2D)  (None, 7, 7, 640)    2457600     ['block7c_se_excite[0][0]']      
                                                                                                  
 block7c_project_bn (BatchNorma  (None, 7, 7, 640)   2560        ['block7c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block7c_d

In [12]:
# telling the model what cost and optimization method to use
model6.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=METRICS
)


In [ ]:
# fit the model

r = model6.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-13-3ee413fc21c8>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model6.fit_generator(


Epoch 1/10
 12/315 [>.............................] - ETA: 22:15 - loss: 12.0559 - tp: 38.0000 - fp: 154.0000 - tn: 806.0000 - fn: 154.0000 - accuracy: 0.7326 - precision: 0.1979 - recall: 0.1979

In [ ]:
model6.save('efficientB7.h5')

# VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model

In [ ]:
Image_size=[224,224]
vgg=VGG19(input_shape=Image_size+[3],weights='imagenet',include_top=False)
vgg.output

In [ ]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
x=Flatten()(vgg.output)
prediction=Dense(6,activation='sigmoid')(x)
model7=Model(inputs=vgg.input,outputs=prediction)
model7.summary()

In [ ]:
# telling the model what cost and optimization method to use
model7.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics='accuracy'
)


In [ ]:
# fit the model

r = model7.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
model7.save('vgg19.h5')

# AlexNet

In [ ]:
# Common lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Utils
from tqdm import tqdm
import datetime

# Sklearn
from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Input
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, Flatten, AveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
# When export model, these preprocessing layers will be saved along with the rest of model
resize_and_rescale = Sequential([
    Resizing(227, 227),
    Rescaling(1./255)
])

In [ ]:
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
        
create_dir('models')

In [ ]:
lamb = 0.9

model = Sequential([
    # Preprocessing layer
    resize_and_rescale,
    
    InputLayer((227, 227, 3)),
    
    # 1st layer
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu', kernel_regularizer=l2(lamb), name='conv1'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'),
    BatchNormalization(),
    
    # 2nd layer
    Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(lamb), name='conv2'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'),
    BatchNormalization(),
    
    # 3rd layer
    Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(lamb), name='conv3'),
    BatchNormalization(),
    
    # 4th layer
    Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(lamb), name='conv4'),
    BatchNormalization(),
    
    # 5th layer
    Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(lamb), name='conv5'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'),
    BatchNormalization(),
    
    # Flatten
    Flatten(),
    
    # 6th layer
    Dense(units=4096, activation='relu'),
    Dropout(0.5),
    
    # 7th layer
    Dense(units=4096, activation='relu'),
    Dropout(0.5),
    
    # 8th layer (output)
    Dense(units=6, activation='softmax')
], name='AlexNet')

In [ ]:
#Create folder contains model's files
model_dir = 'models/alexnet'
model_file = 'best_alexnet.hdf5'
create_dir(model_dir)

checkpoint = ModelCheckpoint(os.path.join(model_dir, model_file),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=30,
                               verbose=1,
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), 
                              patience=7, min_delta=1e-3, verbose=1, min_lr=1e-7)

In [ ]:
# Initialize TensorBoard
log_dir = 'models/alexnet/logs' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# telling the model what cost and optimization method to use
model8.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics='accuracy'
)


In [ ]:
# fit the model

r = model8.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
model8.save('alexNet.h5')

# Xception

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adamax

from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
inputs = tf.keras.layers.Input((150,150,3))
base_model=tf.keras.applications.xception.Xception(include_top=False, weights="imagenet",input_shape=(150,150,3), pooling='avg') 
x=base_model(inputs)
output=layers.Dense(6, activation='softmax')(x)
model9=tf.keras.models.Model(inputs=inputs, outputs=output)

In [ ]:
# telling the model what cost and optimization method to use
model9.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=METRICS
)


In [ ]:
# fit the model

r = model9.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
model9.save('xception.h5')